In [1]:
# https://docs.gpytorch.ai/en/stable/examples/01_Exact_GPs/Simple_GP_Regression.html
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline

The training and test data is sampled from:
$$
y = \sin(2\pi x) + \epsilon \quad \epsilon \sim \mathcal{N}(0,0.04)
$$

In [2]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

A Gaussian process is a way to define distributions over functions of the form $f : \mathcal{X} \rightarrow \mathbb{R}$, where $\mathcal{X}$ is any domain.

A Guassian Process (GP) is collection of dependent random variables, one for each $\mathbf{x} \in \mathcal{X}$, such that every finite subset of which has a multivariate Guassian distribution. A $GP(\mu(\mathbf{x}), k(\mathbf{x}, \mathbf{x}'))$ is specified by its mean function $\mu(\mathbf{x}) = \mathbb{E}[f(\mathbf{x})]$ and covariance (or kernel) function $k(\mathbf{x}, \mathbf{x}') = \mathbb{E}[f(x) - \mu(\mathbf{x})(f(x') - \mu(\mathbf{x}')]$. For GPs not conditioned on data, we assume that $\mu=0$.
The random variables here represent the value of $f(\mathbf{x})$ at $\mathbf{x}$.


In [3]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

GP is a bayesian method, we can estimate the hyperparameters $\theta$ of the kernel by maximizing the marginal likelihood:
$
\log p(\mathbf{y} | \mathbf{X}, \theta)
$
where
$$
p(\mathbf{y} | \mathbf{X}, \theta) = \int p(\mathbf{y} | \mathbf{f}, \mathbf{X}) p(\mathbf{f} | \mathbf{X}, \theta) d \mathbf{f}
$$

In [5]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: -0.059   lengthscale: 0.276   noise: 0.033
Iter 2/50 - Loss: -0.063   lengthscale: 0.301   noise: 0.037
Iter 3/50 - Loss: -0.064   lengthscale: 0.297   noise: 0.040
Iter 4/50 - Loss: -0.063   lengthscale: 0.288   noise: 0.042
Iter 5/50 - Loss: -0.062   lengthscale: 0.283   noise: 0.043
Iter 6/50 - Loss: -0.063   lengthscale: 0.285   noise: 0.042
Iter 7/50 - Loss: -0.064   lengthscale: 0.291   noise: 0.041
Iter 8/50 - Loss: -0.065   lengthscale: 0.299   noise: 0.039
Iter 9/50 - Loss: -0.065   lengthscale: 0.306   noise: 0.038
Iter 10/50 - Loss: -0.064   lengthscale: 0.308   noise: 0.037
Iter 11/50 - Loss: -0.064   lengthscale: 0.306   noise: 0.036
Iter 12/50 - Loss: -0.064   lengthscale: 0.303   noise: 0.036
Iter 13/50 - Loss: -0.064   lengthscale: 0.300   noise: 0.036
Iter 14/50 - Loss: -0.065   lengthscale: 0.299   noise: 0.037
Iter 15/50 - Loss: -0.065   lengthscale: 0.301   noise: 0.038
Iter 16/50 - Loss: -0.065   lengthscale: 0.305   noise: 0.039
Iter 17/50 - Loss



In this case, the posterior distribution is again another Gaussian process.


In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(0, 1, 51)
    observed_pred = likelihood(model(test_x))

f_preds = model(test_x)
y_preds = likelihood(model(test_x))

f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix
#f_samples = f_preds.sample(sample_shape=torch.Size(1000,))

In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(4, 3))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])